In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# nn.LazyLinear是一种延迟初始化的全连接层（线性层），它允许在未指定输入特征维度的情况下创建层（只指定输出维度）
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

In [3]:
# 由于网络不知道输入层权重的维度，所以框架尚未初始化任何参数
net[0].weight

<UninitializedParameter>

In [4]:
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)


In [6]:
# 接下来我们通过网络传递数据，让框架最终初始化参数
X = torch.rand(2, 20)
net(X)

print(net, '\n')
print(net[0].weight.shape)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
) 

torch.Size([256, 20])


In [ ]:
# @装饰器，本质上是一个函数，它可以用来修改或扩展另一个函数或类的行为
# 这里表示将某个方法或属性动态添加到d2l.Module类中
@d2l.add_to_class(d2l.Module)
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

add_to_class的实现如下：
```python
def add_to_class(Class):  # 装饰器的外层函数，是个高阶函数
    """Register functions as methods in created class.

    Defined in :numref:`sec_oo-design`"""
    def wrapper(obj):  # 内部函数，obj是被装饰的对象（通常是一个函数）
        setattr(Class, obj.__name__, obj)  # 将传入的函数obj动态绑定到类Class中（作为类的一个方法）
    return wrapper  # 返回一个装饰器函数
```
当使用@add_to_class(Class)装饰某个函数时，wrapper会被调用，并将该函数添加到指定的类Class中

注：

动态添加的含义：程序在**运行**时，将新的属性或方法添加到类或对象中。
